In [1]:
from dotenv import load_dotenv
import os
from connect_db import *
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
load_dotenv()

True

In [3]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPEN_API_KEY')
llm = ChatOpenAI(temperature=0,model='gpt-3.5-turbo')
db = connect_db()
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(toolkit=toolkit, llm=llm, verbose=True)

In [6]:
# Define a function to generate llm_config from a LangChain tool
def generate_llm_config(tool):
    # Define the function schema based on the tool's args_schema
    function_schema = {
        "name": tool.name.lower().replace (' ', '_'),
        "description": tool.description,
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
        },
    }

    if tool.args is not None:
      function_schema["parameters"]["properties"] = tool.args

    return function_schema

In [8]:
tools = []
functions = {}
for tool in toolkit.get_tools():
    tool_schema = generate_llm_config(tool)
    print(tool_schema)
    tools.append(tool_schema)
    functions[tool.name] = tool._run



{'name': 'sql_db_query', 'description': "Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", 'parameters': {'type': 'object', 'properties': {'query': {'title': 'Query', 'type': 'string'}}, 'required': []}}
{'name': 'sql_db_schema', 'description': "Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: 'table1, table2, table3'", 'parameters': {'type': 'object', 'properties': {'table_names': {'title': 'Table Names', 'type': 'string'}}, 'required': []}}
{'name': 'sql_db_list_tables', 'description': 'Input is an empty string, output is a comma 